In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
import time
import datetime
from datetime import timedelta
import scipy.stats as stats

In [28]:
ID_13_2020=pd.read_csv(r'D:\留德十年\TUD\ADP\Data\2020/TagID_013_2020.csv',dtype= {'CreateDateTime': str, 'TagValue': np.float64, 'TagId': np.int32},parse_dates=['CreateDateTime'],index_col='Unnamed: 0')
ID_14_2020=pd.read_csv(r'D:\留德十年\TUD\ADP\Data\2020/TagID_014_2020.csv',dtype= {'CreateDateTime': str, 'TagValue': np.float64, 'TagId': np.int32},parse_dates=['CreateDateTime'],index_col='Unnamed: 0')
ID_85_2020=pd.read_csv(r'D:\留德十年\TUD\ADP\Data\2020/TagID_085_2020.csv',dtype= {'CreateDateTime': str, 'TagValue': np.float64, 'TagId': np.int32},parse_dates=['CreateDateTime'],index_col='Unnamed: 0')
ID_52_2020=pd.read_csv(r'D:\留德十年\TUD\ADP\Data\2020/TagID_052_2020.csv',dtype= {'CreateDateTime': str, 'TagValue': np.float64, 'TagId': np.int32},parse_dates=['CreateDateTime'],index_col='Unnamed: 0')


In [23]:
#计算时间间隔，将间隔大于2D的超长周期数据挑选出来
def get_index(data):
    a = data
    a['TagValue_1']=a.CreateDateTime.diff(1).dt.total_seconds()
    data_da=a[a['TagValue_1']>172800]
#     data['TagValue_2']=data.TagValue.diff(1)
#     data['TagValue_2'] = data.TagValue_2.map(lambda x:float(x))
#     data_da=data[(data['TagValue_1']>86400)&(data['TagValue_2']>-1)]
#     data_da1=data_da[data_da['TagValue'] == data_da['TagValue_2'] ]
    return data_da

In [24]:
#挑选无效区间
def get_index_useless(data):
    a = data
    a['TagValue_1']=a.CreateDateTime.diff(1).dt.total_seconds()
    a['TagValue_2']=a.TagValue.diff(1)
    a['TagValue_2'] = a.TagValue_2.map(lambda x:float(x))
    data_da=a[(a['TagValue_1']>86400)&(a['TagValue_2'] == 0)]
    data_da1=data_da[data_da['TagValue'] == 0]
    return data_da1

In [14]:
#按照挑选出的无效区间，将用作重采样的52时间序列进行切割
def new_base(data1,data2):
    for i in range(0,len(get_index_useless(data1))):
        a = get_index_useless(data1).reset_index().loc[i]['CreateDateTime']
        b = data1.loc[get_index_useless(data1).reset_index().loc[i]['index']-1]['CreateDateTime']
        c = data2
        c=c[(c['CreateDateTime']>a)|(c['CreateDateTime']<b)]
    return c

In [34]:
#重采样
def up_new(data1,data2):
    a = data1.set_index('CreateDateTime').join(data2.set_index('CreateDateTime'),lsuffix = '_l', rsuffix = '_r',how = 'outer')
    a = a.fillna(method = "ffill")
    data2_resample = a.drop('TagValue_l',axis = 1) 
    data2_resample = data2_resample.drop('TagId_l',axis = 1) 
    return data2_resample

In [5]:
def Pearson(data):
    df = data 
    overall_pearson_r = df.corr().iloc[0,2]
    print(f"Pandas computed Pearson r: {overall_pearson_r}")
    
    
    b = df.columns.tolist()[0]
    c = df.columns.tolist()[2]
    r, p = stats.pearsonr(df.dropna()[b], df.dropna()[c])
    print(f"Scipy computed Pearson r: {r} and p-value: {p}")
    
    
    df2=df[[b,c]]
    f,ax=plt.subplots(figsize=(7,3))
    df2.rolling(window=150,center=True).median().plot(ax=ax)
    ax.set(xlabel='Time',ylabel='Smiling Evidence')
    ax.set(title=f"Overall Pearson r = {np.round(overall_pearson_r,2)}")
    plt.show()

In [35]:
get_index(ID_13_2020)

,CreateDateTime,TagValue,TagId,TagValue_1
1563,2020-02-25 18:23:45.826,1.0,13,436875.588
2956,2020-03-23 07:14:56.304,1.0,13,1391944.201
4086,2020-04-28 14:24:15.771,0.0,13,2212021.386
4412,2020-05-04 08:58:08.906,1.0,13,206463.953
4514,2020-05-11 12:50:15.041,1.0,13,410986.059
4725,2020-06-22 14:07:45.977,0.0,13,3243675.167
6042,2020-07-20 10:25:39.723,0.0,13,1343354.433
6043,2020-07-23 14:10:11.550,0.0,13,272671.827
6048,2020-07-27 08:27:52.284,1.0,13,319514.940
10789,2020-09-21 09:35:13.066,0.0,13,780991.333


In [36]:
get_index_useless(ID_13_2020)

,CreateDateTime,TagValue,TagId,TagValue_1,TagValue_2
4086,2020-04-28 14:24:15.771,0.0,13,2212021.386,0.0
6042,2020-07-20 10:25:39.723,0.0,13,1343354.433,0.0
6043,2020-07-23 14:10:11.550,0.0,13,272671.827,0.0
7361,2020-08-08 13:01:42.365,0.0,13,124699.152,0.0


In [37]:
base = new_base(ID_13_2020,ID_52_2020)

In [38]:
base1 = new_base(ID_13_2020,ID_85_2020)
base2 = new_base(ID_13_2020,ID_13_2020)

In [40]:
a = up_new(base,base1)
b = up_new(base,base2)

In [51]:
a

,TagValue_r,TagId_r
CreateDateTime,,
2020-02-06 13:05:56.982,0.0,85.0
2020-02-06 13:05:58.643,0.0,85.0
2020-02-06 13:05:59.049,0.0,85.0
2020-02-06 13:06:00.073,0.0,85.0
2020-02-06 13:06:00.277,0.0,85.0
...,...,...
2020-12-31 23:08:28.691,0.0,85.0
2020-12-31 23:23:26.096,0.0,85.0
2020-12-31 23:23:26.307,0.0,85.0


In [52]:
c = a.join(b,lsuffix = '_l', rsuffix = '_r',how = 'outer')
c = c.fillna(method = "ffill")


In [53]:
c

,TagValue_r_l,TagId_r_l,TagValue_r_r,TagId_r_r,TagValue_1,TagValue_2
CreateDateTime,,,,,,
2020-02-06 13:05:56.982,0.0,85.0,0.0,13.0,NaN,NaN
2020-02-06 13:05:58.643,0.0,85.0,0.0,13.0,NaN,NaN
2020-02-06 13:05:59.049,0.0,85.0,0.0,13.0,NaN,NaN
2020-02-06 13:06:00.073,0.0,85.0,0.0,13.0,NaN,NaN
2020-02-06 13:06:00.277,0.0,85.0,0.0,13.0,NaN,NaN
...,...,...,...,...,...,...
2020-12-31 23:08:28.691,0.0,85.0,0.0,13.0,15.965,-1.0
2020-12-31 23:23:26.096,0.0,85.0,0.0,13.0,15.965,-1.0
2020-12-31 23:23:26.307,0.0,85.0,0.0,13.0,15.965,-1.0


In [ ]:
df = c 
overall_pearson_r = df.corr().iloc[0,2]
print(f"Pandas computed Pearson r: {overall_pearson_r}")


r, p = stats.pearsonr(df.dropna()['TagValue_r_l'], df.dropna()['TagValue_r_r'])
print(f"Scipy computed Pearson r: {r} and p-value: {p}")


df2=df[['TagValue_r_l','TagValue_r_r']]
f,ax=plt.subplots(figsize=(7,3))
df2.rolling(window=150,center=True).median().plot(ax=ax)
ax.set(xlabel='Time',ylabel='Smiling Evidence')
ax.set(title=f"Overall Pearson r = {np.round(overall_pearson_r,2)}")
plt.show()